Імпортуємо датасет квіточок

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"boomcrouser","key":"9806d26c109d290b0332410399df4c9d"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d atlkor/floversdata
!unzip floversdata.zip

Імортування залежностей

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.utils import image_dataset_from_directory
from keras.callbacks import EarlyStopping

Функція перетворення тензору в новий

In [3]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

Колл-бек припинення навчання, якщо відстежуваний показник val_loss перестав покращуватися

In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

Вказуємо папку із зображеннями

In [5]:
images_dir = '/content/img/img'

Генеруємо датасет для тренування

In [6]:
train_dataset = image_dataset_from_directory(images_dir, image_size=(100,100), seed = 1, batch_size= 32, label_mode='categorical', color_mode='rgb')
train_dataset = train_dataset.map(process)

Found 2011 files belonging to 14 classes.


Створюємо модель

In [7]:
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape = (100, 100, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(14, activation='softmax')
])

Функція оптимізації

In [8]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

Компілюємо модель

In [9]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

Дивимось яка чудова модель вийшла

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 44, 44, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                    

In [ ]:
model.fit(
    train_dataset,
    epochs=20,
    use_multiprocessing = True,
    shuffle = True,
    validation_data=train_dataset,
    validation_split=0.2, callbacks=[early_stopping]
)

Epoch 1/20
63/63 [==============================] - 14s 88ms/step - loss: 2.6237 - accuracy: 0.0865 - val_loss: 2.5835 - val_accuracy: 0.1198
Epoch 2/20
63/63 [==============================] - 3s 44ms/step - loss: 2.5389 - accuracy: 0.1537 - val_loss: 2.4182 - val_accuracy: 0.2561
Epoch 3/20
63/63 [==============================] - 4s 64ms/step - loss: 2.3327 - accuracy: 0.2367 - val_loss: 2.1790 - val_accuracy: 0.3013
Epoch 4/20
63/63 [==============================] - 4s 64ms/step - loss: 2.1432 - accuracy: 0.3043 - val_loss: 2.0254 - val_accuracy: 0.3496
Epoch 5/20
63/63 [==============================] - 4s 68ms/step - loss: 2.0115 - accuracy: 0.3451 - val_loss: 1.8889 - val_accuracy: 0.4008
Epoch 6/20
63/63 [==============================] - 3s 49ms/step - loss: 1.8958 - accuracy: 0.3874 - val_loss: 1.7758 - val_accuracy: 0.4351
Epoch 7/20
63/63 [==============================] - 4s 66ms/step - loss: 1.7996 - accuracy: 0.4227 - val_loss: 1.6576 - val_accuracy: 0.5087
Epoch 8/20
6